# Sistema de asistencia de manejo de un vehículo terrestre a través de visión por computadora V1

In [24]:
import time
import sys
import random
import cv2
import os
from datetime import datetime
from ultralytics import YOLO  # Para cargar el modelo YOLOv8

Cargar las rutas de los otros modulos del sistema

In [25]:
sys.path.append('sensores_input')  # Agregar la ruta del simulador de sensores
sys.path.append('Traffic_Signs_Model')  # Agregar la ruta del modelo YOLOv8

Cargar simulacion de sensores

In [26]:
from simulacionSensores import simulate_speed  # Importar simulador de velocidad

Cargar el modelo YOLOv8 entrenado

In [27]:
model = YOLO('Traffic_Signs_Model/traffic_signs_model.pt')

In [28]:
print(model.names)

{0: 'pedestrian_traffic_light_green', 1: 'pedestrian_traffic_light_red', 2: 'traffic_light_green', 3: 'traffic_light_red', 4: 'traffic_light_yellow', 5: 'stop'}


Definiciones generales de limites de velocidad y objetos peligrosos

In [29]:
SPEED_LIMIT = 80  # Límite de velocidad urbana (km/h)
DANGER_OBJECTS = ['stop', 'traffic_light_red', 'pedestrian_traffic_light_red']  # Objetos que generan alerta

Hacemos un archivo con logs para poder medir y saber como funciona nuestro modelo en su prueba en un entorno simulado

In [30]:
log_filename = 'modelv1_alert_log.txt'
with open(log_filename, 'w') as log_file:
    log_file.write("Alerta Log para ModelV1\n")
    log_file.write("Fecha de creación: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n\n")


Función para la generación de alertas

In [31]:
def generate_alert(object_detected, current_speed):
    if object_detected in DANGER_OBJECTS and current_speed > 0:
        return f"ALERTA: {object_detected}"
    return "No hay alerta."

Función para la detección de objetos frame a frame usando el modelo de YOLOv8

In [32]:
# Función para la detección de objetos frame a frame usando YOLOv8
def detect_objects(frame):
    results = model(frame)  # Pasamos el frame al modelo YOLO
    if results and len(results[0].boxes) > 0:
        # Convertir el tensor de la clase a un entero para poder usarlo como índice
        top_object_idx = int(results[0].boxes[0].cls.item())
        top_object = results[0].names[top_object_idx]
        return top_object
    else:
        return "No objects detected"


Procesar distintos tipos de input

In [33]:
def process_input(input_type, input_source, frame_rate=20):
    cap = None
    if input_type == 'image':
        # Cargar la imagen directamente
        frame = cv2.imread(input_source)
        yield frame, 0  # Se devuelve un timestamp de 0 ya que es una imagen estática

    elif input_type == 'video':
        cap = cv2.VideoCapture(input_source)

    elif input_type == 'camera':
        cap = cv2.VideoCapture(0)  # 0 para la cámara predeterminada

    # Si es un video o la cámara
    if cap:
        current_frame = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break  # Salir si no hay más frames o no se puede leer

            timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)  # Obtener el timestamp en milisegundos

            if input_type == 'video' and current_frame % frame_rate == 0:
                yield frame, timestamp  # Devolver frame y timestamp cada 'frame_rate'

            elif input_type == 'camera':
                yield frame, timestamp  # Devolver frame en tiempo real si es la cámara

            current_frame += 1

        cap.release()

Funcionamiento del modelo en su versión 1

In [34]:
def modelv1(input_type='camera', input_source=None):
    # Inicializar parámetros de simulación de velocidad
    current_speed = 0  # Velocidad inicial del vehículo
    max_speed = 70
    acceleration = 5.0
    deceleration = 7.0
    stop_deceleration = 15.0
    time_between_stops = 7
    stop_chance = 0.1

    # Iterar sobre los frames según el input type
    for frame, timestamp in process_input(input_type, input_source):
        # Simular velocidad
        current_speed = simulate_speed(current_speed, max_speed, acceleration, deceleration, stop_deceleration, time_between_stops, stop_chance)
        # current_speed = 60
        print(f"Velocidad actual: {current_speed} km/h")

        # Mostrar el frame en una ventana
        cv2.imshow("Frame", frame)

        # Procesar detección de objetos
        detected_object = detect_objects(frame)
        print(f"Objeto detectado: {detected_object}")

        # Generar alerta
        alert_message = generate_alert(detected_object, current_speed)
        print(alert_message)

        # Guardar los datos en el archivo de log si hay una alerta
        if "ALERTA" in alert_message:
            with open(log_filename, 'a') as log_file:
                log_file.write(f"Timestamp: {timestamp} ms\n")
                log_file.write(f"Velocidad: {current_speed} km/h\n")
                log_file.write(f"Objeto detectado: {detected_object}\n")
                log_file.write(f"Mensaje de alerta: {alert_message}\n")
                log_file.write(f"-----------------------------\n")

        # Romper el ciclo si se presiona la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Simulación de intervalo de tiempo de 0.1 segundos
        time.sleep(0.1)

    cv2.destroyAllWindows()

In [35]:
input_type = 'video'  # Cambiar entre 'image', 'video', 'camera' según sea necesario
input_source = 'video_input/Recorriendo CIUDAD DE GUATEMALA en vehículo (Parte 1) - GUATEMALA 2024.mp4'  # Ruta del archivo de video


In [36]:
modelv1(input_type, input_source)

Velocidad actual: 0.56 km/h

0: 384x640 2 traffic_light_yellows, 49.6ms
Speed: 1.5ms preprocess, 49.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Objeto detectado: traffic_light_yellow
No hay alerta.
Velocidad actual: 0.81 km/h

0: 384x640 2 traffic_light_greens, 47.3ms
Speed: 1.0ms preprocess, 47.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Objeto detectado: traffic_light_green
No hay alerta.
Velocidad actual: 1.2 km/h

0: 384x640 1 traffic_light_green, 1 traffic_light_yellow, 42.3ms
Speed: 0.9ms preprocess, 42.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Objeto detectado: traffic_light_green
No hay alerta.
Velocidad actual: 1.9 km/h

0: 384x640 1 traffic_light_green, 1 traffic_light_yellow, 42.5ms
Speed: 1.0ms preprocess, 42.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Objeto detectado: traffic_light_green
No hay alerta.
Velocidad actual: 2.08 km/h

0: 384x640 1 traffic_light_green, 41.4ms
